In [ ]:
!git clone https://github.com/ggerganov/llama.cpp.git

Cloning into 'llama.cpp'...
remote: Enumerating objects: 21901, done.
remote: Counting objects: 100% (9654/9654), done.
remote: Compressing objects: 100% (550/550), done.
remote: Total 21901 (delta 9387), reused 9178 (delta 9103), pack-reused 12247
Receiving objects: 100% (21901/21901), 26.33 MiB | 18.17 MiB/s, done.
Resolving deltas: 100% (15503/15503), done.


# Download the finetuned huggingface model

In [ ]:
from huggingface_hub import snapshot_download
model_id="ysr/deepseek-rust-1.3b-lora-4k"
snapshot_download(repo_id=model_id, local_dir="hyperparam-rust",
                  local_dir_use_symlinks=False, revision="main")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/744 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/18.0k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.37M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/5.54k [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/400M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

'/content/hyperparam-rust'

# Install llama.cpp dependencies

In [ ]:
!pip install -r llama.cpp/requirements.txt

## Convert the model in GGUF format and quantize it in 8 bit

In [ ]:
!python llama.cpp/convert.py hyperparam-rust \
  --outfile hyperparam-rust-1.3b-4k.gguf \
  --outtype q8_0 \
  --vocab-type bpe \
  --pad-vocab

Loading model file hyperparam-rust/model-00001-of-00002.safetensors
Loading model file hyperparam-rust/model-00001-of-00002.safetensors
Loading model file hyperparam-rust/model-00002-of-00002.safetensors
params = Params(n_vocab=32256, n_embd=2048, n_layer=24, n_ctx=16384, n_ff=5504, n_head=16, n_head_kv=16, n_experts=None, n_experts_used=None, f_norm_eps=1e-06, rope_scaling_type=<RopeScalingType.LINEAR: 'linear'>, f_rope_freq_base=100000, f_rope_scale=4.0, n_orig_ctx=None, rope_finetuned=None, ftype=<GGMLFileType.MostlyF16: 1>, path_model=PosixPath('hyperparam-rust'))
Loaded vocab file PosixPath('hyperparam-rust/tokenizer.json'), type 'bpe'
Vocab info: <BpeVocab with 32000 base tokens and 22 added tokens>
Special vocab info: <SpecialVocab with 31757 merges, special tokens {'bos': 32013, 'eos': 32014, 'pad': 32014}, add special tokens {'bos': True, 'eos': False}>
Permuting layer 0
Permuting layer 1
Permuting layer 2
Permuting layer 3
Permuting layer 4
Permuting layer 5
Permuting layer 6

# Login to huggingface hub

In [ ]:
import huggingface_hub
huggingface_hub.login()

# Upload the GGUF model to hub :)

In [ ]:
from huggingface_hub import HfApi
api = HfApi()

model_id = "ysr/hyperpara-1.3b-v1-f16-gguf"
api.create_repo(model_id, exist_ok=True, repo_type="model")
api.upload_file(
    path_or_fileobj="hyperparam-rust-1.3b-4k.gguf",
    path_in_repo="hyperparam-rust-1.3b-4k-f16.gguf",
    repo_id=model_id,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


hyperparam-rust-1.3b-4k.gguf:   0%|          | 0.00/2.69G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ysr/hyperpara-1.3b-v1-f16-gguf/commit/4a59087e374a9c71ceef0e4a3da455c2baaad739', commit_message='Upload hyperparam-rust-1.3b-4k-f16.gguf with huggingface_hub', commit_description='', oid='4a59087e374a9c71ceef0e4a3da455c2baaad739', pr_url=None, pr_revision=None, pr_num=None)